In [1]:
import os
import ctypes
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path as pt
from ipywidgets import widgets
import time as t
from numba import jit

#%matplotlib qt5
%matplotlib widget

In [2]:
import numpy as np
import Kinetic_Simualtion
from Class_Definitions import Ions
from Class_Definitions import Conditions
from Class_Definitions import Ratecoefficients
from Class_Definitions import Time
from Class_Definitions import Fonts
from Class_Definitions import Rates


In [3]:
I = Ions()
CD = Conditions()
R = Ratecoefficients()
RA = Rates()
T = Time()

In [4]:
conditions = []
def simulation_NoVariation():
    global conditions
    
    density = 5e14
    a = 0.5
    P_THz = 2e-5
    TrapTime = 600
    Temp = 5.7
    TIon = 12.3
    mIon = 14
    
    LIICG_signal = np.zeros(1, dtype=float)
    conditions = [P_THz, 0.5, TrapTime, Temp, density, a, TIon, mIon]
    # conditions[P_THZ[W], branching ratio CD_0/CD_1 backward reaction, trap time [ms], trap temp[K], He density [cm^-3], a]
    Kinetic_Simualtion.simulation(conditions, RA, R, CD, I, T, 1)
    
    LIICG_signal[0] = I.LIICG
    print(f"{LIICG_signal=}")


In [5]:
%%time
simulation_NoVariation()

R.B_10=1.0860637725122443e+19, R.B_10=1.0860637725122443e+19
sigma=304445.2999049122, gamma=219134.66179497942, norm=8.008104937907556e-07, 1.0684865111459952e-15
34813.334735209966
B01: 3.48e+04, B10: 1.16e+04
11604.444911736655
0.000624
0.24000000000000002
0.12000000000000001
0.725
0.335
0.9500000000000002
1.42e+03, 2.16e+04
9.19e-01, 1.73e+04
5.24e+01, 6.51e+04
LIICG_signal=array([22.47105012])
Wall time: 3.57 s


In [6]:
Energy = [0, 15.127861, 45.373851, 90.718526, 151.132755, 226.577764]  # typ in energy levels in cm^-1 (CDMS)
g = [1, 3, 5, 7, 9, 11]

KT  = 0.695035*CD.temp
R.q_10 = 1*4.3242e-11
R.q_01 = R.q_10 * (g[1]*np.exp(-Energy[1]/KT)) / (g[0]*np.exp(-Energy[0]/KT))

RA.Rate_q_01 = R.q_01*CD.n_He
RA.Rate_q_10 = R.q_10*CD.n_He
print(f"{RA.Rate_q_01=:.2e}, {RA.Rate_q_10=:.2e}"), R.q_10

RA.Rate_q_01=1.42e+03, RA.Rate_q_10=2.16e+04


(None, 4.3242e-11)

In [7]:
g[1]*np.exp(-Energy[1]/KT), g[0]*np.exp(-Energy[0]/KT)

(0.06588029831452646, 1.0)

In [8]:
np.exp(-Energy[1]/KT), Energy[1], KT, 0.695035*CD.temp, 0.695035*5.7

(0.02196009943817549, 15.127861, 3.9616995, 3.9616995, 3.9616995)

In [9]:
Noff = [I.CD_0_off, I.CD_1_off, I.CD_2_off, I.CDHe_off, I.CDHe2_off]
Non = [I.CD_0_on, I.CD_1_on, I.CD_2_on, I.CDHe_on, I.CDHe2_on]
Noff, Non[0].shape

([array([938.14493343, 938.14380785, 938.14268626, ..., 825.92374172,
         825.92293793, 825.92213415]),
  array([61.80526808, 61.8052308 , 61.80518955, ..., 54.41306086,
         54.41300791, 54.41295496]),
  array([0.04979775, 0.04979775, 0.04979773, ..., 0.04384164, 0.0438416 ,
         0.04384155]),
  array([0.00000000e+00, 1.16285708e-03, 2.32570663e-03, ...,
         9.96910422e+01, 9.96916322e+01, 9.96922223e+01]),
  array([0.00000000e+00, 0.00000000e+00, 4.21535692e-09, ...,
         1.99283129e+01, 1.99285796e+01, 1.99288463e+01])],
 (120000,))

In [10]:
mol = "CD"
tag = "He"

fig, ax = plt.subplots()

lg = [f"{mol}(0)", f"{mol}(1)", f"{mol}(2)", f"{mol}{tag}", f"{mol}{tag}2"]
c = 0

x = np.linspace(0, 600, 120000)
for off, on in zip(Noff, Non):
    ax.plot(x, off, f"C{c}-", label=f"{lg[c]}")
    ax.plot(x, on, f"C{c}--")
    c += 1
    
ax.set_yscale("log")
ax.legend(title="- Off, -- On")
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
from scipy.constants import Boltzmann as k_boltzmann, speed_of_light as C, Planck, m_p
from scipy.special import wofz
epsilon = 8.854e-12
trap_area = 5.5e-5
reduced_plank = Planck / (2*np.pi)
reduced_plank

1.0545718176461565e-34

In [65]:
gauss_fwhm = lambda freq, mass, temp: freq * np.sqrt( (8*k_boltzmann*temp*np.log(2)) / (m_p*mass*C**2))
lorrentz_fwhm = lambda dipole, power: ( dipole*3.33564e-30 ) /( reduced_plank*np.pi*np.sqrt(C*epsilon*trap_area) ) * np.sqrt(power)

faddava_func = lambda lorrentz, gaussian: 1j*lorrentz/(gaussian*np.sqrt(2))
voigt = lambda lorrentz, gaussian: (wofz(faddava_func(lorrentz, gaussian)) / (np.sqrt(2*np.pi) * gaussian)).real

freq = 453521850000
sigma = gauss_fwhm(freq, conditions[7], conditions[6])
print(sigma, freq*np.sqrt(8*conditions[6]*k_boltzmann*np.log(2)/C**2/conditions[7]/m_p))

gamma = lorrentz_fwhm(1.313, conditions[0])
print(f"{gamma=:.3e}, {4.9e7*np.sqrt(conditions[0]):.3e}" )
norm = voigt(gamma, sigma)
norm

303365.7332556174 303365.7332556174
gamma=1.547e+05, 2.191e+05


9.136475856579267e-07

In [19]:
T.num_steps
T.step
T.timestep

5e-06

In [20]:
T.step = 5e-3      # set timestep in ms
T.timestep = T.step/1000  # converting step in s
T.num_steps = int(CD.time/T.step)  # number of steps for the simulation


In [22]:
int(CD.time/T.step)

120000

In [30]:
time = np.linspace(0, 0.6, T.num_steps)
yoff = [I.CD_0_off, I.CD_1_off, I.CD_2_off, I.CDHe_off, I.CDHe2_off]
yoff = np.array(yoff)

fig, ax = plt.subplots()
ax.plot(time, yoff.T)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[(&#39;Home&#39;, &#39;Reset original view&#39;, &#39;home&#39;, &#39;home&#39;), (&#39;Back&#39;, &#39;Back to previous …

In [23]:
I.CD_0_off.shape

(120000,)

In [32]:
np.array([1, 2, 3])+[1, 2, 3]

array([2, 4, 6])

In [34]:
600/5e-3R

120000.0